In [1]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

Loading diamonds dataset...
Dataset loaded successfully!


In [2]:
# Q1 - "How extensive is our inventory database?"
n_rows = df.height
n_cols = df.width
print(f"Our inventory contains {n_rows} diamonds with {n_cols} attributes.")


Our inventory contains 6000 diamonds with 8 attributes.


In [3]:
# Q2 - "What information do we track about each diamond?"
print("\nColumns & dtypes:")
for name, dtype in df.schema.items():
    print(f"  - {name}: {dtype}")


Columns & dtypes:
  - Carat Weight: Float64
  - Cut: String
  - Color: String
  - Clarity: String
  - Polish: String
  - Symmetry: String
  - Report: String
  - Price: Int64


In [4]:
# Q3 - "When I look at one line in this database, what am I looking at?"
print("\nSample records:")
print(df.head(5))


Sample records:
shape: (5, 8)
┌──────────────┬───────┬───────┬─────────┬────────┬──────────┬────────┬───────┐
│ Carat Weight ┆ Cut   ┆ Color ┆ Clarity ┆ Polish ┆ Symmetry ┆ Report ┆ Price │
│ ---          ┆ ---   ┆ ---   ┆ ---     ┆ ---    ┆ ---      ┆ ---    ┆ ---   │
│ f64          ┆ str   ┆ str   ┆ str     ┆ str    ┆ str      ┆ str    ┆ i64   │
╞══════════════╪═══════╪═══════╪═════════╪════════╪══════════╪════════╪═══════╡
│ 1.1          ┆ Ideal ┆ H     ┆ SI1     ┆ VG     ┆ EX       ┆ GIA    ┆ 5169  │
│ 0.83         ┆ Ideal ┆ H     ┆ VS1     ┆ ID     ┆ ID       ┆ AGSL   ┆ 3470  │
│ 0.85         ┆ Ideal ┆ H     ┆ SI1     ┆ EX     ┆ EX       ┆ GIA    ┆ 3183  │
│ 0.91         ┆ Ideal ┆ E     ┆ SI1     ┆ VG     ┆ VG       ┆ GIA    ┆ 4370  │
│ 0.83         ┆ Ideal ┆ G     ┆ SI1     ┆ EX     ┆ EX       ┆ GIA    ┆ 3171  │
└──────────────┴───────┴───────┴─────────┴────────┴──────────┴────────┴───────┘


In [6]:
# Section 2 - Strategic Business Questions
# Question 4: "We're considering expanding our storage facilities. How many total carats do we have in inventory?
# I need to know if our vaults can handle the weight."

# Q4: Total carats
total_carats = df.select(pl.col("Carat Weight").sum()).item()
print(f"Total carats in inventory: {total_carats:.2f}")

Total carats in inventory: 8007.12


In [7]:
# Q5: Variety of cuts
unique_cuts = df.select(pl.col("Cut").n_unique()).item()
print(f"We have {unique_cuts} unique cuts in the inventory.")

We have 5 unique cuts in the inventory.


In [8]:
# Q6: Most & least valuable diamonds
max_price = df.select(pl.col("Price").max()).item()
min_price = df.select(pl.col("Price").min()).item()
print(f"Most valuable diamond: ${max_price:,}")
print(f"Least valuable diamond: ${min_price:,}")

Most valuable diamond: $101,561
Least valuable diamond: $2,184


In [10]:
# Q7: Typical price by color
avg_price_by_color = (
    df.group_by("Color")
      .agg(pl.col("Price").mean().alias("avg_price"))
      .sort("Color")
)

# avg_price into currency strings for easy read
avg_price_by_color = avg_price_by_color.with_columns(
    pl.col("avg_price").map_elements(lambda x: f"${x:,.2f}", return_dtype=pl.Utf8)
)

print(avg_price_by_color)

shape: (6, 2)
┌───────┬────────────┐
│ Color ┆ avg_price  │
│ ---   ┆ ---        │
│ str   ┆ str        │
╞═══════╪════════════╡
│ D     ┆ $15,255.78 │
│ E     ┆ $11,539.19 │
│ F     ┆ $12,712.24 │
│ G     ┆ $12,520.05 │
│ H     ┆ $10,487.35 │
│ I     ┆ $8,989.64  │
└───────┴────────────┘


In [11]:
# Section 3: Advanced Strategic Analysis
# Q8: Breakdown by clarity
clarity_breakdown = (
    df.group_by("Clarity")
      .agg([
          pl.len().alias("diamond_count"),
          pl.col("Price").mean().alias("avg_price"),
          pl.col("Carat Weight").sum().alias("total_carats")
      ])
      .sort("Clarity")
)

# avg_price as currency
clarity_breakdown = clarity_breakdown.with_columns(
    pl.col("avg_price").map_elements(lambda x: f"${x:,.2f}", return_dtype=pl.Utf8)
)

print(clarity_breakdown)

shape: (7, 4)
┌─────────┬───────────────┬────────────┬──────────────┐
│ Clarity ┆ diamond_count ┆ avg_price  ┆ total_carats │
│ ---     ┆ ---           ┆ ---        ┆ ---          │
│ str     ┆ u32           ┆ str        ┆ f64          │
╞═════════╪═══════════════╪════════════╪══════════════╡
│ FL      ┆ 4             ┆ $63,776.00 ┆ 7.87         │
│ IF      ┆ 219           ┆ $22,105.84 ┆ 316.0        │
│ SI1     ┆ 2059          ┆ $8,018.86  ┆ 2563.69      │
│ VS1     ┆ 1192          ┆ $13,694.11 ┆ 1682.74      │
│ VS2     ┆ 1575          ┆ $11,809.05 ┆ 2170.81      │
│ VVS1    ┆ 285           ┆ $16,845.68 ┆ 389.43       │
│ VVS2    ┆ 666           ┆ $14,142.18 ┆ 876.58       │
└─────────┴───────────────┴────────────┴──────────────┘


In [12]:
# Q9: Pricing structure across cut-color
cut_color_prices = (
    df.group_by(["Cut", "Color"])
      .agg(pl.col("Price").mean().alias("avg_price"))
      .sort(["Cut", "Color"])
)

# avg_price as currency
cut_color_prices = cut_color_prices.with_columns(
    pl.col("avg_price").map_elements(lambda x: f"${x:,.2f}", return_dtype=pl.Utf8)
)

print(cut_color_prices)


shape: (30, 3)
┌───────────┬───────┬────────────┐
│ Cut       ┆ Color ┆ avg_price  │
│ ---       ┆ ---   ┆ ---        │
│ str       ┆ str   ┆ str        │
╞═══════════╪═══════╪════════════╡
│ Fair      ┆ D     ┆ $6,058.25  │
│ Fair      ┆ E     ┆ $5,370.62  │
│ Fair      ┆ F     ┆ $6,063.62  │
│ Fair      ┆ G     ┆ $7,345.52  │
│ Fair      ┆ H     ┆ $5,908.50  │
│ …         ┆ …     ┆ …          │
│ Very Good ┆ E     ┆ $12,101.91 │
│ Very Good ┆ F     ┆ $12,413.91 │
│ Very Good ┆ G     ┆ $12,354.01 │
│ Very Good ┆ H     ┆ $10,056.11 │
│ Very Good ┆ I     ┆ $8,930.03  │
└───────────┴───────┴────────────┘


In [13]:
# Q10: Top 5 most valuable inventory segments
top_segments = (
    df.group_by(["Color", "Cut"])
      .agg([
          pl.len().alias("diamond_count"),
          pl.col("Price").sum().alias("total_value")
      ])
      .sort("total_value", descending=True)
      .head(5)
)

# Format total_value as currency
top_segments = top_segments.with_columns(
    pl.col("total_value").map_elements(lambda x: f"${x:,.2f}", return_dtype=pl.Utf8)
)

print(top_segments)


shape: (5, 4)
┌───────┬───────────┬───────────────┬───────────────┐
│ Color ┆ Cut       ┆ diamond_count ┆ total_value   │
│ ---   ┆ ---       ┆ ---           ┆ ---           │
│ str   ┆ str       ┆ u32           ┆ str           │
╞═══════╪═══════════╪═══════════════╪═══════════════╡
│ G     ┆ Ideal     ┆ 690           ┆ $9,363,514.00 │
│ G     ┆ Very Good ┆ 578           ┆ $7,140,620.00 │
│ F     ┆ Very Good ┆ 455           ┆ $5,648,327.00 │
│ F     ┆ Ideal     ┆ 363           ┆ $5,346,782.00 │
│ H     ┆ Ideal     ┆ 458           ┆ $5,279,687.00 │
└───────┴───────────┴───────────────┴───────────────┘
